# Cleaning `condition` feature

Notebook used for cleaning and visualizing the `condition` column in the Drug Review dataset.

First, installation of `pyxet` and other dependencies:

In [1]:
%pip install pyxet
%pip install polars
%pip install pandas
%pip install numpy
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyxet         # make xet:// protocol available   
import pandas as pd 
import polars as pl  # faster alternative to pandas
import numpy as np
import pyarrow
import matplotlib.pyplot as plt

fs = pyxet.XetFS()

# Drug Review Dataset (UCI)

In [7]:
# get the drug training and testing data
df_drugs_train = pl.read_csv('xet://alt2177/mllm-data/main/data/drug_data/drugsComTrain_raw.tsv', separator = '\t')
df_drugs_test = pl.read_csv('xet://alt2177/mllm-data/main/data/drug_data/drugsComTest_raw.tsv', separator = '\t')
print(df_drugs_train.head())

shape: (5, 7)
┌────────┬────────────────┬────────────────┬────────────────┬────────┬───────────────┬─────────────┐
│        ┆ drugName       ┆ condition      ┆ review         ┆ rating ┆ date          ┆ usefulCount │
│ ---    ┆ ---            ┆ ---            ┆ ---            ┆ ---    ┆ ---           ┆ ---         │
│ i64    ┆ str            ┆ str            ┆ str            ┆ f64    ┆ str           ┆ i64         │
╞════════╪════════════════╪════════════════╪════════════════╪════════╪═══════════════╪═════════════╡
│ 206461 ┆ Valsartan      ┆ Left           ┆ "It has no     ┆ 9.0    ┆ May 20, 2012  ┆ 27          │
│        ┆                ┆ Ventricular    ┆ side effect, I ┆        ┆               ┆             │
│        ┆                ┆ Dysfunction    ┆ take i…        ┆        ┆               ┆             │
│ 95260  ┆ Guanfacine     ┆ ADHD           ┆ "My son is     ┆ 8.0    ┆ April 27,     ┆ 192         │
│        ┆                ┆                ┆ halfway        ┆        ┆ 2010  

In [8]:
# get the shape of our data
print("Training Data shape: {}".format(df_drugs_train.shape))
print("Testing Data shape: {}".format(df_drugs_test.shape))

Training Data shape: (161297, 7)
Testing Data shape: (53766, 7)


## `condition` column

### Training Data

In [9]:
df_drugs_train["condition"].describe()

statistic,value
str,i64
"""count""",161297
"""null_count""",899
"""unique""",885


In [14]:
# fill missing values with "not_reported"
df_filled = df_drugs_train.with_columns(
    pl.col("condition").fill_null(pl.lit("not_reported")),
)

# check if we no longer have null values
print(df_filled.null_count())

shape: (1, 7)
┌─────┬──────────┬───────────┬────────┬────────┬──────┬─────────────┐
│     ┆ drugName ┆ condition ┆ review ┆ rating ┆ date ┆ usefulCount │
│ --- ┆ ---      ┆ ---       ┆ ---    ┆ ---    ┆ ---  ┆ ---         │
│ u32 ┆ u32      ┆ u32       ┆ u32    ┆ u32    ┆ u32  ┆ u32         │
╞═════╪══════════╪═══════════╪════════╪════════╪══════╪═════════════╡
│ 0   ┆ 0        ┆ 0         ┆ 0      ┆ 0      ┆ 0    ┆ 0           │
└─────┴──────────┴───────────┴────────┴────────┴──────┴─────────────┘


In [21]:
# set the number of rows we want to see
pl.Config.set_tbl_rows(100)

# get only the condition column
print(df_filled.select("condition").unique())

shape: (885, 1)
┌───────────────────────────────────┐
│ condition                         │
│ ---                               │
│ str                               │
╞═══════════════════════════════════╡
│ High Blood Pressure               │
│ Organ Transplant, Rejection Reve… │
│ Tuberculosis, Active              │
│ Endometrial Hyperplasia           │
│ Lyme Disease, Neurologic          │
│ 32</span> users found this comme… │
│ Infectious Diarrhea               │
│ Coccidioidomycosis                │
│ Ovarian Cance                     │
│ Acetaminophen Overdose            │
│ 14</span> users found this comme… │
│ Breast Cancer, Prevention         │
│ Seizure Prevention                │
│ Gout, Acute                       │
│ 72</span> users found this comme… │
│ Subarachnoid Hemorrhage           │
│ Smoking Cessation                 │
│ Diabetes Insipidus                │
│ Lipodystrophy                     │
│ Deep Vein Thrombosis Prophylaxis… │
│ acial Lipoatrophy               

There are many entries with html tags like `</span>` inside, which should not be showing up in our data.

In [22]:
# removing html tags
df_condition = df_filled.filter(~pl.col("condition").str.contains("</span>"))


In [ ]:
# df_drugs_train.dtypes  #= [Int64, String, String, String, Float64, String, Int64]
df_drugs_train.describe()
df_drugs_train.select(pl.all().is_null().sum())

#df_drugs_train.select(['rating']).unique()
# for cond in df_drugs_train.select(['condition']).unique()['condition']:
#     print(cond)
# for drug in df_drugs_train.select(['drugName']).unique()['drugName']:
#     print(drug)
# for rev in df_drugs_train.select(['review']).unique()['review'][0:20]:
#     print(rev)

review_word_count = df_drugs_train.select(['review']).map_rows(lambda t: len(t[0].split(" ")))
#plt.hist(review_word_count)
review_word_count.describe()
plt.hist(df_drugs_train['rating'], bins = 10)
plt.hist(df_drugs_train['usefulCount'], bins = 100)
#df_drugs_train.select(['usefulCount']).unique()


In [1]:
print("garbage")

garbage


Getting the basic descriptive stats, data types, etc.

In [33]:
# Get descripive stats
df_drugs_train.describe()

describe,,drugName,condition,review,rating,date,usefulCount
str,f64,str,str,str,f64,str,f64
"""count""",161297.0,"""161297""","""161297""","""161297""",161297.0,"""161297""",161297.0
"""null_count""",0.0,"""0""","""899""","""0""",0.0,"""0""",0.0
"""mean""",115923.585305,null,null,null,6.994377,null,28.004755
"""std""",67004.44517,null,null,null,3.272329,null,36.403742
"""min""",2.0,"""A + D Cracked …","""0</span> users…",""""" please t…",1.0,"""April 1, 2008""",0.0
"""25%""",58063.0,null,null,null,5.0,null,6.0
"""50%""",115744.0,null,null,null,8.0,null,16.0
"""75%""",173776.0,null,null,null,10.0,null,36.0
"""max""",232291.0,"""femhrt""","""zen Shoulde""","""""❤️❤️❤️ Cialis…",10.0,"""September 9, 2…",1291.0


In [34]:
# get column names and their dtypes
df_drugs_train.schema

OrderedDict([('', Int64),
             ('drugName', Utf8),
             ('condition', Utf8),
             ('review', Utf8),
             ('rating', Float64),
             ('date', Utf8),
             ('usefulCount', Int64)])

In [25]:
# check if all the rows are unique 
print(df_drugs_train.is_unique().all())

True


In [35]:
# check for nulls
df_drugs_train.null_count()

,drugName,condition,review,rating,date,usefulCount
u32,u32,u32,u32,u32,u32,u32
0,0,899,0,0,0,0


Since there are over 800 `null` values in `condition`, let's take a closer look. This column represents the medical conditions of respondents, so we cannot just drop those. Instead, we fill with `not_reported`. This ensures no null values.

In [51]:
# fill missing values with "not_reported"
df_filled = df_drugs_train.with_columns(
    pl.col("condition").fill_null(pl.lit("not_reported")),
)

# check if we no longer have null values
print(df_filled.null_count())

shape: (1, 7)
┌─────┬──────────┬───────────┬────────┬────────┬──────┬─────────────┐
│     ┆ drugName ┆ condition ┆ review ┆ rating ┆ date ┆ usefulCount │
│ --- ┆ ---      ┆ ---       ┆ ---    ┆ ---    ┆ ---  ┆ ---         │
│ u32 ┆ u32      ┆ u32       ┆ u32    ┆ u32    ┆ u32  ┆ u32         │
╞═════╪══════════╪═══════════╪════════╪════════╪══════╪═════════════╡
│ 0   ┆ 0        ┆ 0         ┆ 0      ┆ 0      ┆ 0    ┆ 0           │
└─────┴──────────┴───────────┴────────┴────────┴──────┴─────────────┘


### Testing Data